<a href="https://colab.research.google.com/github/iambrookedrake/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/Brooke_Drake_DSPT6_U2S3M2_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [1]:
##From source:: https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip

--2020-06-08 23:22:48--  https://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66136 (65K) [application/x-httpd-php]
Saving to: ‘Absenteeism_at_work_AAA.zip’

Absenteeism_at_work 100%[===================>]  64.59K   248KB/s    in 0.3s    

2020-06-08 23:22:49 (248 KB/s) - ‘Absenteeism_at_work_AAA.zip’ saved [66136/66136]



In [2]:
! unzip Absenteeism_at_work_AAA.zip

Archive:  Absenteeism_at_work_AAA.zip
  inflating: Absenteeism_at_work.arff  
  inflating: Absenteeism_at_work.csv  
  inflating: Absenteeism_at_work.xls  
  inflating: Attribute Information.docx  
  inflating: UCI_ABS_TEXT.docx       


In [39]:
import pandas as pd
absent = pd.read_excel('Absenteeism_at_work.xls')
print(absent.shape)
absent.head()

(740, 21)


,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239554,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239554,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239554,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239554,97,0,1,2,1,0,1,90,172,30,2


In [40]:
absent['Absenteeism time in hours'].value_counts().sort_index() #But no one requests 0 hours off!

0       44
1       88
2      157
3      112
4       60
5        7
7        1
8      208
16      19
24      16
32       6
40       7
48       1
56       2
64       3
80       3
104      1
112      2
120      3
Name: Absenteeism time in hours, dtype: int64

In [41]:
##Continue to clean and explore your data. 
absent = absent.rename(columns={'Son': 'Kids'})
absent['Vices'] = absent['Social drinker'] + absent['Social smoker'] + absent['Pet']
absent['Overweight'] = absent['Body mass index']>=25
absent['WorkToTravel'] = (8-absent['Absenteeism time in hours'])/(2*absent['Distance from Residence to Work']/60)
absent['WorkToAge'] = (absent['Service time'] + (absent['Education']-1))/(absent['Age'])
absent['OneDayOnly'] = absent['Absenteeism time in hours'] <=8
absent['HalfDayOnly'] = absent['Absenteeism time in hours'] <=5
print(absent.shape)
absent.head()

(740, 27)


,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Kids,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours,Vices,Overweight,WorkToTravel,WorkToAge,OneDayOnly,HalfDayOnly
0,11,26,7,3,1,289,36,13,33,239554,97,0,1,2,1,0,1,90,172,30,4,2,True,3.333333,0.393939,True,True
1,36,0,7,3,1,118,13,18,50,239554,97,1,1,1,1,0,0,98,178,31,0,1,True,18.461538,0.360000,True,True
2,3,23,7,4,1,179,51,18,38,239554,97,0,1,0,1,0,0,89,170,31,2,1,True,3.529412,0.473684,True,True
3,7,7,7,5,1,279,5,14,39,239554,97,0,1,2,1,1,0,68,168,24,4,2,False,24.000000,0.358974,True,True
4,11,23,7,5,1,289,36,13,33,239554,97,0,1,2,1,0,1,90,172,30,2,2,True,5.000000,0.393939,True,True


In [42]:
absent['Disciplinary failure'].value_counts(normalize=True).sort_index()

0    0.945946
1    0.054054
Name: Disciplinary failure, dtype: float64

In [43]:
absent['Month of absence'].value_counts(normalize=True).sort_index()

0     0.004054
1     0.067568
2     0.097297
3     0.117568
4     0.071622
5     0.086486
6     0.072973
7     0.090541
8     0.072973
9     0.071622
10    0.095946
11    0.085135
12    0.066216
Name: Month of absence, dtype: float64

In [44]:
absent = absent[absent['Absenteeism time in hours']>0] #because no one requests 0 hours off
absent = absent.drop(columns=['Month of absence','Work load Average/day ','Hit target',
                              'Disciplinary failure','Social drinker',
                              'Social smoker','Pet','Weight','Height',
                              'Body mass index','Service time','Age', 
                              'Day of the week','Seasons','Transportation expense',
                              'Distance from Residence to Work','Education'], axis=1)
print(absent.shape)
absent.head()

(696, 10)


,ID,Reason for absence,Kids,Absenteeism time in hours,Vices,Overweight,WorkToTravel,WorkToAge,OneDayOnly,HalfDayOnly
0,11,26,2,4,2,True,3.333333,0.393939,True,True
2,3,23,0,2,1,True,3.529412,0.473684,True,True
3,7,7,2,4,2,False,24.000000,0.358974,True,True
4,11,23,2,2,2,True,5.000000,0.393939,True,True
5,3,23,0,2,1,True,3.529412,0.473684,True,True


In [45]:
absent['OneDayOnly'].value_counts(normalize=True)

True     0.909483
False    0.090517
Name: OneDayOnly, dtype: float64

In [80]:
##For the evaluation metric you chose, what score would you get just by guessing?
print(f'Original Guess from baseline MEAN:')
print(absent['HalfDayOnly'].value_counts(normalize=True)[1])
print()
absent['HalfDayOnly'].value_counts(normalize=True)

Original Guess from baseline MEAN:
0.6091954022988506



True     0.609195
False    0.390805
Name: HalfDayOnly, dtype: float64

In [47]:
absent['ID'].value_counts()

3     112
28     75
34     54
22     46
20     39
11     38
15     35
14     29
24     29
36     28
10     24
33     23
1      22
17     20
18     14
5      14
13     13
25     10
6       8
9       8
12      7
27      7
30      6
32      5
23      5
26      5
7       4
29      4
2       4
31      2
21      2
16      2
19      2
Name: ID, dtype: int64

In [48]:
absent['ID'].value_counts().describe()

count     33.000000
mean      21.090909
std       23.867294
min        2.000000
25%        5.000000
50%       13.000000
75%       29.000000
max      112.000000
Name: ID, dtype: float64

In [49]:
import numpy as np
table = pd.pivot_table(absent, values='Absenteeism time in hours', index=['ID'],
                    columns=['HalfDayOnly'], aggfunc=np.sum, fill_value=0)

table

HalfDayOnly,False,True
ID,,
1,88,33
2,24,1
3,272,210
5,96,8
6,72,0
7,24,6
9,256,6
10,175,11
11,432,18


In [50]:
total = table
total.reset_index(level=total.index.names, inplace=True)
total['DayParts']=(0.5*total[True])+total[False]
total['Liklihood']=(total['DayParts']/total['DayParts'].mean())
total.sort_values(by=['DayParts'],ascending=False)


HalfDayOnly,ID,False,True,DayParts,Liklihood
11,14,440,36,458.0,3.273200
8,11,432,18,441.0,3.151705
2,3,272,210,377.0,2.694315
31,34,248,96,296.0,2.115430
32,36,272,39,291.5,2.083270
25,28,200,147,273.5,1.954629
17,20,224,82,265.0,1.893882
6,9,256,6,259.0,1.851002
21,24,232,22,243.0,1.736654
12,15,184,69,218.5,1.561559


In [52]:
requests = pd.DataFrame(total['ID'])
requests['Liklihood'] = total['Liklihood']
requests

,ID,Liklihood
0,1,0.746833
1,2,0.175095
2,3,2.694315
3,5,0.714672
4,6,0.514564
5,7,0.192962
6,9,1.851002
7,10,1.289984
8,11,3.151705
9,12,0.207255


In [53]:
possible = absent.merge(requests)
possible.shape

,ID,Reason for absence,Kids,Absenteeism time in hours,Vices,Overweight,WorkToTravel,WorkToAge,OneDayOnly,HalfDayOnly,Liklihood
0,11,26,2,4,2,True,3.333333,0.393939,True,True,3.151705
1,11,23,2,2,2,True,5.000000,0.393939,True,True,3.151705
2,11,18,2,8,2,True,0.000000,0.393939,True,False,3.151705
3,11,23,2,1,2,True,5.833333,0.393939,True,True,3.151705
4,11,18,2,8,2,True,0.000000,0.393939,True,False,3.151705


In [55]:
possible.tail(10)

,ID,Reason for absence,Kids,Absenteeism time in hours,Vices,Overweight,WorkToTravel,WorkToAge,OneDayOnly,HalfDayOnly,Liklihood
686,12,19,1,8,9,False,0.000000,0.064516,True,False,0.207255
687,12,22,1,8,9,False,0.000000,0.064516,True,False,0.207255
688,12,22,1,3,9,False,2.941176,0.064516,True,True,0.207255
689,12,19,1,2,9,False,3.529412,0.064516,True,True,0.207255
690,29,28,2,2,3,True,12.000000,0.439024,True,True,0.132214
691,29,19,2,3,3,True,10.000000,0.439024,True,True,0.132214
692,29,14,2,8,3,True,0.000000,0.439024,True,False,0.132214
693,29,22,2,8,3,True,0.000000,0.439024,True,False,0.132214
694,16,23,2,8,2,True,0.000000,0.521739,True,False,0.114348
695,16,7,2,8,2,True,0.000000,0.521739,True,False,0.114348


In [56]:
possible = possible.drop(columns=['ID','Absenteeism time in hours'], axis=1)
possible.shape

(696, 9)

In [78]:
###Can you make a fast, first model that beats guessing?
from sklearn.linear_model import LinearRegression
model = LinearRegression()

target = 'HalfDayOnly'
features = ['OneDayOnly','Liklihood', 'Kids', 'Vices', 'Overweight', 'WorkToTravel',
            'WorkToAge']##Try adding 'Reason for absence'
X_train = possible[features]
y_train = possible[target]

model.fit(X_train, y_train)

X_test = [[True, 1, 0,0,False,possible['WorkToTravel'].mean(),possible['WorkToAge'].mean()]]
y_pred = model.predict(X_test)

print(f'Original Guess from baseline MEAN:')
print(absent['HalfDayOnly'].value_counts(normalize=True)[1])
print(f'Linear Model-- Chance Request Off is Only Half Day:')
print(y_pred)

Original Guess from baseline MEAN:
0.6091954022988506
Linear Model-- Chance Request Off is Only Half Day:
[0.79693693]
